In [ ]:
#| default_exp api

In [1]:
#| export
from Cloud2DImageConverter import spherical_projection as sp
from Cloud2DImageConverter import image_generator as ig
from Cloud2DImageConverter import data
from tqdm import tqdm
from PIL import Image
import numpy as np
import shutil
import os

In [ ]:
#| export
def run(data_path, results_path, batch_size=500, fov_up=3.0, fov_down=-25.0, width=1024, height=64, is_label=True):
    velodyne_path = data_path+"velodyne"
    label_path = data_path + "labels" if is_label else None
    max_len = len(os.listdir(velodyne_path))
    if batch_size > max_len: 
        batch_size = max_len-1

    if not os.path.exists(results_path): 
        os.makedirs(results_path)
        os.makedirs(results_path+"reflectance")
        os.makedirs(results_path+"depth")
        if is_label:
            os.makedirs(results_path+"segmentation_mask")
        print(f"folder created at {results_path}")

    for batch in tqdm(range(batch_size, max_len, batch_size), desc="Batch:"):
        start, end = ig.define_range(batch, batch_size, max_len)
        velodyne_list = sorted(os.listdir(velodyne_path))[start:end]
        label_list = sorted(os.listdir(label_path))[start:end] if is_label else None
        point_cloud = data.load_data(velodyne_path, velodyne_list, label_path, label_list)
        projection_dict = ig.do_projection(point_cloud, fov_up, fov_down, width, height, is_label)
        ig.create_images(projection_dict, results_path)

In [ ]:
#| export
def color_matrix(matrix):
    matrix = np.vectorize(data.learning_map_inv.get)(matrix)
    colored_matrix =  np.empty(matrix.shape + (3,), dtype=np.uint8)
    color_map = data.color_map.items()
    for key, value in color_map:
        indices = np.where(matrix == key)
        colored_matrix[indices] = value
    matrix = colored_matrix
    return matrix

Merge reflectance and depth images.\
Args:
- origin_path: path to the folder containing _reflectance_ and _depth_ folders
- destiny_path: path to save the merged images

In [10]:
#| export
def merge_images(origin_path, destiny_path):
    reflectance_path =  origin_path+"reflectance/"
    depth_path = origin_path+"depth/"

    if not os.path.exists(destiny_path): 
        os.makedirs(destiny_path)
        print(f"destiny folder created at {destiny_path}")

    for file_name in os.listdir(reflectance_path):
        reflectance_image = Image.open(reflectance_path+file_name)
        depth_image = Image.open(depth_path+file_name)

        merged_image = np.dstack((np.array(reflectance_image), np.array(depth_image)))
        merged_image = Image.fromarray(merged_image)

        output_path = os.path.join(destiny_path, file_name)
        merged_image.save(output_path)

In [ ]:
#| export
def split_aux(splited_list):
    splited_array = np.array(splited_list)
    splited_array = splited_array.reshape(16, 1024)
    img = Image.fromarray(splited_array)
    return img
    
def split_image(image):
    list_1, list_2, list_3, list_4 = [[] for _ in range(4)]
    for index, line in enumerate(np.array(image)):
        if index % 4 == 0:
            list_1.append(list(line))
        if index % 4 == 1:
            list_2.append(list(line))
        if index % 4 == 2:
            list_3.append(list(line))
        if index % 4 == 3:
            list_4.append(list(line))

    img_1 = split_aux(list_1)
    img_2 = split_aux(list_2)
    img_3 = split_aux(list_3)
    img_4 = split_aux(list_4)

    return img_1, img_2, img_3, img_4

In [11]:
#| hide
import nbdev; nbdev.nbdev_export()